# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [51]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [52]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [53]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [54]:
encoded[:100]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70])

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [55]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [56]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr) // characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[: characters_per_batch * n_batches]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [57]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [58]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[31 64 57 72 76 61 74  1 16  0]
 [ 1 57 69  1 70 71 76  1 63 71]
 [78 65 70 13  0  0  3 53 61 75]
 [70  1 60 77 74 65 70 63  1 64]
 [ 1 65 76  1 65 75 11  1 75 65]
 [ 1 37 76  1 79 57 75  0 71 70]
 [64 61 70  1 59 71 69 61  1 62]
 [26  1 58 77 76  1 70 71 79  1]
 [76  1 65 75 70  7 76 13  1 48]
 [ 1 75 57 65 60  1 76 71  1 64]]

y
 [[64 57 72 76 61 74  1 16  0  0]
 [57 69  1 70 71 76  1 63 71 65]
 [65 70 13  0  0  3 53 61 75 11]
 [ 1 60 77 74 65 70 63  1 64 65]
 [65 76  1 65 75 11  1 75 65 74]
 [37 76  1 79 57 75  0 71 70 68]
 [61 70  1 59 71 69 61  1 62 71]
 [ 1 58 77 76  1 70 71 79  1 75]
 [ 1 65 75 70  7 76 13  1 48 64]
 [75 57 65 60  1 76 71  1 64 61]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [135]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name="inputs")
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
        
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [134]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    def build_cell(lstm_size, keep_prob):
        
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell outputs
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        
        return drop
    

    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for i in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)

    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [131]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output, axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size,out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.add(tf.matmul(x, softmax_w), softmax_b)
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name="predictions")
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [62]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [63]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [137]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss =  build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [138]:
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [139]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 1...  Training loss: 4.4181...  0.3325 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.3215...  0.2605 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 3.7683...  0.2598 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 5.1363...  0.2620 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 4.1916...  0.2650 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 4.0400...  0.2625 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 3.8214...  0.2625 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 3.6313...  0.2600 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 3.5144...  0.2630 sec/batch
Epoch: 1/20...  Training Step: 10...  Training loss: 3.4753...  0.2660 sec/batch
Epoch: 1/20...  Training Step: 11...  Training loss: 3.4038...  0.2555 sec/batch
Epoch: 1/20...  Training Step: 12...  Training loss: 3.4114...  0.2550 sec/batch
Epoch: 1/20...  Training Step: 13... 

Epoch: 1/20...  Training Step: 103...  Training loss: 3.1104...  0.2795 sec/batch
Epoch: 1/20...  Training Step: 104...  Training loss: 3.1000...  0.2755 sec/batch
Epoch: 1/20...  Training Step: 105...  Training loss: 3.1009...  0.2800 sec/batch
Epoch: 1/20...  Training Step: 106...  Training loss: 3.1020...  0.2685 sec/batch
Epoch: 1/20...  Training Step: 107...  Training loss: 3.0766...  0.2690 sec/batch
Epoch: 1/20...  Training Step: 108...  Training loss: 3.0813...  0.2595 sec/batch
Epoch: 1/20...  Training Step: 109...  Training loss: 3.0887...  0.2820 sec/batch
Epoch: 1/20...  Training Step: 110...  Training loss: 3.0592...  0.2725 sec/batch
Epoch: 1/20...  Training Step: 111...  Training loss: 3.0805...  0.2715 sec/batch
Epoch: 1/20...  Training Step: 112...  Training loss: 3.0753...  0.2641 sec/batch
Epoch: 1/20...  Training Step: 113...  Training loss: 3.0669...  0.2877 sec/batch
Epoch: 1/20...  Training Step: 114...  Training loss: 3.0512...  0.2745 sec/batch
Epoch: 1/20...  

Epoch: 2/20...  Training Step: 203...  Training loss: 2.4809...  0.2710 sec/batch
Epoch: 2/20...  Training Step: 204...  Training loss: 2.4842...  0.2645 sec/batch
Epoch: 2/20...  Training Step: 205...  Training loss: 2.4888...  0.2555 sec/batch
Epoch: 2/20...  Training Step: 206...  Training loss: 2.4750...  0.2710 sec/batch
Epoch: 2/20...  Training Step: 207...  Training loss: 2.4955...  0.2711 sec/batch
Epoch: 2/20...  Training Step: 208...  Training loss: 2.4582...  0.2720 sec/batch
Epoch: 2/20...  Training Step: 209...  Training loss: 2.4666...  0.2571 sec/batch
Epoch: 2/20...  Training Step: 210...  Training loss: 2.4757...  0.2690 sec/batch
Epoch: 2/20...  Training Step: 211...  Training loss: 2.4650...  0.2680 sec/batch
Epoch: 2/20...  Training Step: 212...  Training loss: 2.4975...  0.2738 sec/batch
Epoch: 2/20...  Training Step: 213...  Training loss: 2.4632...  0.2482 sec/batch
Epoch: 2/20...  Training Step: 214...  Training loss: 2.4668...  0.2627 sec/batch
Epoch: 2/20...  

Epoch: 2/20...  Training Step: 303...  Training loss: 2.2359...  0.2680 sec/batch
Epoch: 2/20...  Training Step: 304...  Training loss: 2.2504...  0.2815 sec/batch
Epoch: 2/20...  Training Step: 305...  Training loss: 2.2389...  0.2650 sec/batch
Epoch: 2/20...  Training Step: 306...  Training loss: 2.2581...  0.2610 sec/batch
Epoch: 2/20...  Training Step: 307...  Training loss: 2.2504...  0.2775 sec/batch
Epoch: 2/20...  Training Step: 308...  Training loss: 2.2281...  0.2730 sec/batch
Epoch: 2/20...  Training Step: 309...  Training loss: 2.2416...  0.2625 sec/batch
Epoch: 2/20...  Training Step: 310...  Training loss: 2.2499...  0.2640 sec/batch
Epoch: 2/20...  Training Step: 311...  Training loss: 2.2268...  0.2700 sec/batch
Epoch: 2/20...  Training Step: 312...  Training loss: 2.2227...  0.2705 sec/batch
Epoch: 2/20...  Training Step: 313...  Training loss: 2.2239...  0.2635 sec/batch
Epoch: 2/20...  Training Step: 314...  Training loss: 2.1847...  0.2620 sec/batch
Epoch: 2/20...  

Epoch: 3/20...  Training Step: 403...  Training loss: 2.0866...  0.2640 sec/batch
Epoch: 3/20...  Training Step: 404...  Training loss: 2.0925...  0.2556 sec/batch
Epoch: 3/20...  Training Step: 405...  Training loss: 2.1163...  0.2605 sec/batch
Epoch: 3/20...  Training Step: 406...  Training loss: 2.0791...  0.2620 sec/batch
Epoch: 3/20...  Training Step: 407...  Training loss: 2.0689...  0.2555 sec/batch
Epoch: 3/20...  Training Step: 408...  Training loss: 2.0584...  0.2510 sec/batch
Epoch: 3/20...  Training Step: 409...  Training loss: 2.0782...  0.2615 sec/batch
Epoch: 3/20...  Training Step: 410...  Training loss: 2.1248...  0.2686 sec/batch
Epoch: 3/20...  Training Step: 411...  Training loss: 2.0653...  0.2580 sec/batch
Epoch: 3/20...  Training Step: 412...  Training loss: 2.0602...  0.2645 sec/batch
Epoch: 3/20...  Training Step: 413...  Training loss: 2.0692...  0.2580 sec/batch
Epoch: 3/20...  Training Step: 414...  Training loss: 2.1124...  0.2725 sec/batch
Epoch: 3/20...  

...

Epoch: 18/20...  Training Step: 3559...  Training loss: 1.2385...  0.2440 sec/batch
Epoch: 18/20...  Training Step: 3560...  Training loss: 1.2298...  0.2420 sec/batch
Epoch: 18/20...  Training Step: 3561...  Training loss: 1.2139...  0.2430 sec/batch
Epoch: 18/20...  Training Step: 3562...  Training loss: 1.2380...  0.2470 sec/batch
Epoch: 18/20...  Training Step: 3563...  Training loss: 1.2280...  0.2431 sec/batch
Epoch: 18/20...  Training Step: 3564...  Training loss: 1.2327...  0.2435 sec/batch
Epoch: 19/20...  Training Step: 3565...  Training loss: 1.3512...  0.2415 sec/batch
Epoch: 19/20...  Training Step: 3566...  Training loss: 1.2546...  0.2416 sec/batch
Epoch: 19/20...  Training Step: 3567...  Training loss: 1.2361...  0.2470 sec/batch
Epoch: 19/20...  Training Step: 3568...  Training loss: 1.2648...  0.2440 sec/batch
Epoch: 19/20...  Training Step: 3569...  Training loss: 1.2136...  0.2495 sec/batch
Epoch: 19/20...  Training Step: 3570...  Training loss: 1.2028...  0.2445 se

Epoch: 19/20...  Training Step: 3657...  Training loss: 1.2121...  0.2510 sec/batch
Epoch: 19/20...  Training Step: 3658...  Training loss: 1.2038...  0.2495 sec/batch
Epoch: 19/20...  Training Step: 3659...  Training loss: 1.2176...  0.2465 sec/batch
Epoch: 19/20...  Training Step: 3660...  Training loss: 1.2494...  0.2445 sec/batch
Epoch: 19/20...  Training Step: 3661...  Training loss: 1.2401...  0.2475 sec/batch
Epoch: 19/20...  Training Step: 3662...  Training loss: 1.2049...  0.2465 sec/batch
Epoch: 19/20...  Training Step: 3663...  Training loss: 1.2238...  0.2460 sec/batch
Epoch: 19/20...  Training Step: 3664...  Training loss: 1.2186...  0.2495 sec/batch
Epoch: 19/20...  Training Step: 3665...  Training loss: 1.2378...  0.2420 sec/batch
Epoch: 19/20...  Training Step: 3666...  Training loss: 1.2219...  0.2510 sec/batch
Epoch: 19/20...  Training Step: 3667...  Training loss: 1.2336...  0.2405 sec/batch
Epoch: 19/20...  Training Step: 3668...  Training loss: 1.2260...  0.2495 se

Epoch: 19/20...  Training Step: 3755...  Training loss: 1.1952...  0.2450 sec/batch
Epoch: 19/20...  Training Step: 3756...  Training loss: 1.2250...  0.2511 sec/batch
Epoch: 19/20...  Training Step: 3757...  Training loss: 1.2283...  0.2585 sec/batch
Epoch: 19/20...  Training Step: 3758...  Training loss: 1.2102...  0.2620 sec/batch
Epoch: 19/20...  Training Step: 3759...  Training loss: 1.1970...  0.2505 sec/batch
Epoch: 19/20...  Training Step: 3760...  Training loss: 1.2363...  0.2625 sec/batch
Epoch: 19/20...  Training Step: 3761...  Training loss: 1.2213...  0.2525 sec/batch
Epoch: 19/20...  Training Step: 3762...  Training loss: 1.2141...  0.2695 sec/batch
Epoch: 20/20...  Training Step: 3763...  Training loss: 1.3467...  0.2595 sec/batch
Epoch: 20/20...  Training Step: 3764...  Training loss: 1.2448...  0.2665 sec/batch
Epoch: 20/20...  Training Step: 3765...  Training loss: 1.2294...  0.2780 sec/batch
Epoch: 20/20...  Training Step: 3766...  Training loss: 1.2628...  0.2535 se

Epoch: 20/20...  Training Step: 3853...  Training loss: 1.2057...  0.2550 sec/batch
Epoch: 20/20...  Training Step: 3854...  Training loss: 1.2111...  0.2525 sec/batch
Epoch: 20/20...  Training Step: 3855...  Training loss: 1.2044...  0.2580 sec/batch
Epoch: 20/20...  Training Step: 3856...  Training loss: 1.1947...  0.2580 sec/batch
Epoch: 20/20...  Training Step: 3857...  Training loss: 1.2057...  0.2650 sec/batch
Epoch: 20/20...  Training Step: 3858...  Training loss: 1.2427...  0.2630 sec/batch
Epoch: 20/20...  Training Step: 3859...  Training loss: 1.2234...  0.2675 sec/batch
Epoch: 20/20...  Training Step: 3860...  Training loss: 1.1903...  0.2540 sec/batch
Epoch: 20/20...  Training Step: 3861...  Training loss: 1.2114...  0.2465 sec/batch
Epoch: 20/20...  Training Step: 3862...  Training loss: 1.2012...  0.2560 sec/batch
Epoch: 20/20...  Training Step: 3863...  Training loss: 1.2292...  0.2540 sec/batch
Epoch: 20/20...  Training Step: 3864...  Training loss: 1.2177...  0.2495 se

Epoch: 20/20...  Training Step: 3951...  Training loss: 1.2228...  0.2510 sec/batch
Epoch: 20/20...  Training Step: 3952...  Training loss: 1.2241...  0.2585 sec/batch
Epoch: 20/20...  Training Step: 3953...  Training loss: 1.1907...  0.2500 sec/batch
Epoch: 20/20...  Training Step: 3954...  Training loss: 1.1912...  0.2490 sec/batch
Epoch: 20/20...  Training Step: 3955...  Training loss: 1.2199...  0.2535 sec/batch
Epoch: 20/20...  Training Step: 3956...  Training loss: 1.2013...  0.2470 sec/batch
Epoch: 20/20...  Training Step: 3957...  Training loss: 1.1786...  0.2480 sec/batch
Epoch: 20/20...  Training Step: 3958...  Training loss: 1.2190...  0.2510 sec/batch
Epoch: 20/20...  Training Step: 3959...  Training loss: 1.2127...  0.2470 sec/batch
Epoch: 20/20...  Training Step: 3960...  Training loss: 1.2023...  0.2495 sec/batch


#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [128]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints\\i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i600_l512.ckpt"

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [120]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [121]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [129]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints\\i600_l512.ckpt'

In [130]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints\i600_l512.ckpt
Far"dnHfey
ARJ CCCuuuPPIRiC"i icciE:umR1nIv1zf)f))c
IzCiRmxyIq1Jmnu':yCWCCF/?J1f_1cff!t1z1zfHZciT/CFCM"M1JIc1cI1tHHfyyemc
mHJRrxyyIqnJPyuI/1xntIciTc%m?4IzF_`cFHfaNf
11qIcCucCJMMJC11zH1zft_F1czecy@1zHHf)uAcZ
IH14zfzHuT
1xIetuTJOTyfCCJRMy4 m1ecumZIcCFM%4HHuec
mHcCumr"1Izz1czHmecFMMtC1mm1ezucCIcCiEx4vt4z1vztzf?gu1gz1zz1zzHueyyff'Acim"4z1HtuTuT&FCMCCFJJMCC11et1zzucu Zxc"xurZI4iIucxPT%%pIpilv/p/HueXIDIKJcss/I/i/iRcs&IHH4i1cH)cF33ettv;g
CCF"J@11t1ztt1zf?ettCFf3?edz1c@H1z1fHvB'Rnegg(CJHR G
1HvPcJmRrymngu1z11tvtucZ"xIcZc&mHuTumF"J1znHfoAcZ&IcJm&/I1c1m1Hte'RZcJ"mncCIkv(&/1zumFCJMR"mneci1cumZIqI)cb%///i1/1zuzHukc&It4Iki//Rc/m/Ie1cJttuTumx?qCuRRr" i1imnuRZ"gn2zfq1JRCum"umRCJ RCC@"CJJJm/my33zv1cf!ev1tuzcx"HHlecxm
AEZxH&4uHPXKSSR*ssxusxx*SStZv%XuX/XHPuXk/pITsPPTC%CHiCrIG1iimvaEXyuRiTu/XiCuRCMC"@CHHuRGiRcyuuuuRCJCRRR""iiiRcyumvuRZIqC%2CI44m1t1ztHf?eye ic

IzCHPR%44HnfejcJCCaR""qI2nu'TCImJy&mf13tHf!f
A1"eqnf!d
iTeOCZ"xHPP%xrrHyyItuc

In [126]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l512.ckpt
Far, i;(i; wji9''WC-i)1

qixE5QQ--mgu1'i';Cii)
)
''r1Xu'xmu)1Xm-
jrrrQiiZ1vi''WG!u')xmi)X
pqxZ5Q,
'OxtlZXgQTi''MoXl
gig''WC51
vmu'rxx
Rxp-J-m(m g(e
)i'rx5tX
vnn'iWCX9@
gq?1

w(ncmxx pZ,AQ/Z/gI81('gxziZ5tmi'1Mtmu'1Mv('rxoU)1-m(m
XirmA; mii11' wn'$Cm-Z1uQ'i'Wztm,ui;@mmZE
p
-mCmu9A
-
g rZgEQQ1Qg-

pi7xxgAZoATT22''g(JZZEQ5JIWm giZzzQ; 'WMzpl-'?
wplr1Qg-rcBi'11vi;cii'?km
gj11Lu''WCmuZZQQ-Z-mi'1
'
m-)
Xmm
m
Xir1-
m  jrrc1Qimx,kiZA5pgos20
m

mor1*
miirA)-mgE)
)ii1Cgm- zpr1tm gi'')tX
gcmtQiZ11vgud ii'11Xm

mcrx-
mi9CA
 g(Z
pgoAQQA/,LliIL8I zn Xi)Cm? )
ri9x
m(rBr1'; gl$?
g(m1v

w(rBZZ?u' Gi'1M
gmuZmmu9Cz-'?
wi?cx

 irGQQQ11u
m
rcoZ1mu m
mco11m

wnnZmmE'r1tgu? mme1vm 'iMtm? m(mm-
giZC-z-'r, ipx5pgoQPA;, giZCzi''ttmi'c1-
icxr-goAZQiZgI0m p(Xo'1Mm- rlrCg?
g(g gm-m-QiZCt-'rCt-tq--QUQ-Qz4u'i1v--i'xz-'r1
v(mr-g-X

pxxrZEoAZ, ml(giT2';te eZeeQ)AQ_00g
 mnrC-oi)CtA)'rou1vt
gi9*mgATi!;gi'1VtpZ-'rg-ggTX1L' wml1tm
gi'c1X-  mnZrCE0

mg

mi

In [116]:
checkpoint = 'checkpoints/i1700_l1024.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1700_l1024.ckpt


InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [512,83] rhs shape= [1024,83]
	 [[Node: save/Assign_16 = Assign[T=DT_FLOAT, _class=["loc:@softmax/Variable"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](softmax/Variable/Adam_1, save/RestoreV2_16/_5)]]

Caused by op 'save/Assign_16', defined at:
  File "C:\Users\frank\Anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\frank\Anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\frank\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\frank\Anaconda3\envs\tensorflow-gpu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\frank\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\kerne

In [83]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1200_l512.ckpt
Farse on asanting toot to him on the momer was of ais an ortess, an  to came and torstion. Ant thit hus at the moms of the sureter was tonde hiteng histanten an hose a someniog, het at toe sure her
to the poict an ont he distal sa he was to that
tor thother. He soid at
a praise hored to a sand that at the hadsed ot that
hester to to the heass of the samt and her
about thome her was setiolles her the herse he wos dor ald showing sim ho her
andoters and thaththe care thet whenthat at she has brak sail ho his anest tatt him he was the might as tha pestang of the
rinsor tion, and
toms on thi seaness, wat hee ald nothing tems on hes whate and the mot thom he had,then se hes heed,
and son thene af to mor affome tooniter, and see hod beer taik ng her
sere and thinkse whe her bue siming ofta caiting on has baition of sheideniste that
with she pust ne had book sement of the peosirg. And a seltes the pactestoof to a thes the p